## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [1]:
import os
import string
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re
import pandas as pd
mystem = Mystem()
morph = MorphAnalyzer()

In [2]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

FileNotFoundError: [Errno 2] No such file or directory: 'fpath.txt'

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = './Friends/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [3]:
main_dir = './Friends/'
files_list = []

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов

for root, dirs, files in os.walk(main_dir):
    for name in files:
        files_list.append(os.path.join(root, name))
        #print(os.path.join(root, name))


In [4]:
len(files_list)

165

In [5]:
files_list[:5]

['./Friends/Friends - season 1\\Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt',
 './Friends/Friends - season 1\\Friends - 1x02 - The One With The Sonogram At The End.ru.txt',
 './Friends/Friends - season 1\\Friends - 1x03 - The One With The Thumb.ru.txt',
 './Friends/Friends - season 1\\Friends - 1x04 - The One With George Stephanopoulos.ru.txt',
 './Friends/Friends - season 1\\Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt']

In [6]:
punct = string.punctuation + '«»—…“”'
punct = re.sub("'", '', punct)
table = str.maketrans({ch: None for ch in punct})

In [7]:
content = []

def file_parser():
    for fl in files_list:
        with open(fl, 'r', encoding='UTF-8') as f:  
            text = f.read()
            text_ = re.sub('\n', ' ', text)
            #print(text_)
            tokenize(text)


def tokenize(text):
    text = re.sub('\ufeff', '', text)
    good_tokens = [word for word in word_tokenize(text) if len(word) > 1 
                                                    and not all([ch in punct for ch in word])]
    lemms = ' '.join([morph.parse(token)[0].normal_form for token in good_tokens])
    content.append(lemms)
    
    
    
    

n = file_parser()   

In [8]:
names = []
for name in files_list:
    names.append(name[39:])

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
vectorizer = CountVectorizer(min_df=1)
X = vectorizer.fit(content)
nam = vectorizer.get_feature_names()
X = vectorizer.transform(content)
X.toarray()

array([[5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
df = pd.DataFrame(X.toarray(), index=names, columns=nam).T
term_doc_matrix = df.drop(nam[:180], axis=0)
series = names
tokens = nam[180:]
term_doc_matrix.head()

,1x01 - The One Where Monica Gets A Roommate.ru.txt,1x02 - The One With The Sonogram At The End.ru.txt,1x03 - The One With The Thumb.ru.txt,1x04 - The One With George Stephanopoulos.ru.txt,1x05 - The One With The East German Laundry Detergent.ru.txt,1x06 - The One With The Butt.ru.txt,1x07 - The One With The Blackout.ru.txt,1x08 - The One Where Nana Dies Twice.ru.txt,1x09 - The One Where Underdog Gets Away.ru.txt,1x10 - The One With The Monkey.ru.txt,...,7x15 - The One With Joey's New Brain.ru.txt,7x16 - The One With The Truth About London.ru.txt,7x17 - The One With The Cheap Wedding Dress.ru.txt,7x18 - The One With Joey's Award.ru.txt,7x19 - The One With Ross And Monica's Cousin.ru.txt,7x20 - The One With Rachel's Big Kiss.ru.txt,7x21 - The One With The Vows.ru.txt,7x22 - The One With Chandler's Dad.ru.txt,7x23 - The One With Chandler And Monica's Wedding (1).ru.txt,7x24-25 - The One With Chandler And Monica's Wedding (2).ru.txt
after,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
again,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ahh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
all,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
and,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
tokens

['after',
 'again',
 'ahh',
 'all',
 'and',
 'are',
 'au',
 'aнгел',
 'ban',
 'bay',
 'behind',
 'bodington',
 'bonjour',
 'book',
 'by',
 'careless',
 'cat',
 'cd',
 'cheese',
 'china',
 'chuck',
 'city',
 'co',
 'com',
 'cамый',
 'cочельник',
 'cтарый',
 'cтоп',
 'dais',
 'dance',
 'daysailer',
 'delivery',
 'dickin',
 'dicky',
 'did',
 'dings',
 'discovery',
 'done',
 'doobies',
 'duh',
 'dьzenleme',
 'else',
 'empire',
 'enchante',
 'english',
 'espn',
 'eve',
 'evill',
 'exchanging',
 'face',
 'feel',
 'field',
 'friday',
 'fuchsia',
 'gap',
 'garden',
 'gelula',
 'gentleman',
 'get',
 'girl',
 'glances',
 'glow',
 'gon',
 'goody',
 'got',
 'heart',
 'heт',
 'hipno',
 'hisses',
 'hitchhik',
 'honey',
 'http',
 'hе',
 'ice',
 'iii',
 'in',
 'inc',
 'info',
 'innokenty',
 'it',
 'janice',
 'joey',
 'juliandr',
 'just',
 'khakis',
 'lady',
 'lauren',
 'look',
 'lot',
 'lover',
 'luck',
 'machine',
 'maid',
 'man',
 'mauve',
 'maк',
 'maрша',
 'mb',
 'me',
 'michelle',
 'mik4562000',


In [12]:
term_doc_matrix[2850:2860]

,1x01 - The One Where Monica Gets A Roommate.ru.txt,1x02 - The One With The Sonogram At The End.ru.txt,1x03 - The One With The Thumb.ru.txt,1x04 - The One With George Stephanopoulos.ru.txt,1x05 - The One With The East German Laundry Detergent.ru.txt,1x06 - The One With The Butt.ru.txt,1x07 - The One With The Blackout.ru.txt,1x08 - The One Where Nana Dies Twice.ru.txt,1x09 - The One Where Underdog Gets Away.ru.txt,1x10 - The One With The Monkey.ru.txt,...,7x15 - The One With Joey's New Brain.ru.txt,7x16 - The One With The Truth About London.ru.txt,7x17 - The One With The Cheap Wedding Dress.ru.txt,7x18 - The One With Joey's Award.ru.txt,7x19 - The One With Ross And Monica's Cousin.ru.txt,7x20 - The One With Rachel's Big Kiss.ru.txt,7x21 - The One With The Vows.ru.txt,7x22 - The One With Chandler's Dad.ru.txt,7x23 - The One With Chandler And Monica's Wedding (1).ru.txt,7x24-25 - The One With Chandler And Monica's Wedding (2).ru.txt
джо,0,0,4,0,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
джоана,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джоани,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джовить,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джодить,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джоевый,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джозеф,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джозефина,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джой,6,0,0,3,3,2,1,0,3,1,...,0,0,0,0,0,0,0,0,0,0
джокер,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
### постройте матрицу терм-документ
#term_doc_matrix = []

search = term_doc_matrix.T
### напишите функцию булева поиска по построенной матрице
def boolean_search(text) -> list:
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """
    data = search
    inp = text.split('&')
    for smt in inp:
        smt = smt.lower()
        if smt[0] != '(':
            smt = smt.strip(' ')
            if smt in data.keys():
                data = data[data[smt] > 0]
        else:
            smt = smt.strip('()')
            if smt[0] == 'н':
                word = smt.split()[1]
                word = word.strip(' )(')
                data = data[data[word] == 0]
            else:
                smt = smt.split(' или ')
                data = data[data[smt[0] or smt[1]] > 0]
                   
    return data.index.values.tolist()[:5]


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)'
]
for el in input_text:
    f = boolean_search(el)
    print(el)
    for ser in f:
        print(ser)

Моника & Фиби & Рэйчел & Чендлер & Джои & Росс
2x06 - The One With The Baby On The Bus.ru.txt
5x23-24 - The One In Vegas (1).ru.txt
6x03 - The One With Ross's Denial.ru.txt
6x06 - The One On The Last Night.ru.txt
6x22 - The One Where Paul's The Man.ru.txt
(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс
1x01 - The One Where Monica Gets A Roommate.ru.txt
1x02 - The One With The Sonogram At The End.ru.txt
1x04 - The One With George Stephanopoulos.ru.txt
1x05 - The One With The East German Laundry Detergent.ru.txt
1x07 - The One With The Blackout.ru.txt
(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)
5x24-25 - The One In Vegas (2).ru.txt
6x19 - The One With Joey's Fridge.ru.txt


<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [14]:
from collections import defaultdict

In [15]:
def inverted_index() -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    search = term_doc_matrix.T
    invert = defaultdict(list)
    for el in tokens:
        line = search[search[el] > 0]
        for ser in line.T:
            invert[el].append(ser)
    return invert

inv = inverted_index()

In [17]:
inv

defaultdict(list,
            {'after': ['7x09 - The One With All The Candy.ru.txt'],
             'again': ["5x19 - The One Where Ross Can't Flirt.ru.txt"],
             'ahh': ['5x02 - The One With All The Kissing.ru.txt'],
             'all': ['5x18 - The One Where Rachel Smokes.ru.txt',
              '6x01 - The One After Vegas.ru.txt'],
             'and': ['4x01 - The One With The Jellyfish.Tv.ru.txt',
              "6x19 - The One With Joey's Fridge.ru.txt"],
             'are': ['4x01 - The One With The Jellyfish.Tv.ru.txt'],
             'au': ["5x19 - The One Where Ross Can't Flirt.ru.txt"],
             'aнгел': ['7x13 - The One Where Rosita Dies.ru.txt'],
             'ban': ["7x23 - The One With Chandler And Monica's Wedding (1).ru.txt"],
             'bay': ["7x02 - The One With Rachel's Book.ru.txt"],
             'behind': ['6x12 - The One With The Joke.ru.txt'],
             'bodington': ['5x02 - The One With All The Kissing.ru.txt'],
             'bonjour': ["7x01 - T

С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


In [16]:
for key in inv:
    if len(inv[key]) == 165:
        print(key)

быть
весь
да
думать
если
ещё
знать
как
мой
мочь
мы
на
не
нет
но
ну
он
она
просто
сказать
так
такой
то
тот
ты
хотеть
что
это
этот


In [24]:
one, two, tree, four, five, six, seven = 0, 0, 0, 0, 0, 0, 0
for ser in inv['моника']:
    if int(ser[:1]) == 1:
        one += 1
    if int(ser[:1]) == 2:
        two += 1
    if int(ser[:1]) == 3:
        tree += 1
    if int(ser[:1]) == 4:
        four += 1
    if int(ser[:1]) == 5:
        five += 1
    if int(ser[:1]) == 6:
        six += 1
    if int(ser[:1]) == 7:
        seven += 1

print('Первый: ' + str(one),
      'Второй: ' + str(two),
      'Третий: ' + str(tree),
      'Четвёртый: ' + str(four),
      'Пятый: ' + str(five),
      'Шестой: ' + str(six),
      'Седьмой: ' + str(seven)
     )

Первый: 16 Второй: 24 Третий: 19 Четвёртый: 19 Пятый: 22 Шестой: 22 Седьмой: 23


In [25]:
one, two, tree, four, five, six, seven = 0, 0, 0, 0, 0, 0, 0
for ser in inv['чендлер']:
    if int(ser[:1]) == 1:
        one += 1
    if int(ser[:1]) == 2:
        two += 1
    if int(ser[:1]) == 3:
        tree += 1
    if int(ser[:1]) == 4:
        four += 1
    if int(ser[:1]) == 5:
        five += 1
    if int(ser[:1]) == 6:
        six += 1
    if int(ser[:1]) == 7:
        seven += 1

print('Первый: ' + str(one),
      'Второй: ' + str(two),
      'Третий: ' + str(tree),
      'Четвёртый: ' + str(four),
      'Пятый: ' + str(five),
      'Шестой: ' + str(six),
      'Седьмой: ' + str(seven)
     )

Первый: 0 Второй: 3 Третий: 3 Четвёртый: 4 Пятый: 6 Шестой: 9 Седьмой: 7


## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [ ]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return 

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [ ]:
def compute_sim() -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return 


def get_search_result() -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    return 